In [ ]:
!pip install -q datasets
!pip install -q git+https://github.com/huggingface/transformers.git@add-model-idefics
!pip install -q bitsandbytes sentencepiece accelerate loralib
!pip install -q -U git+https://github.com/huggingface/peft.git

In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from PIL import Image
from transformers import IdeficsForVisionText2Text, AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig

In [ ]:
device = "cude" if torch.cuda.is_available() else "cput"

In [ ]:
checkpoint = "HuggingFaceM4/idefics-9b"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_skip_modules=["lm_head", "embed_tokens"]
)

In [ ]:
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, quantization_config=bnb_config, device_map="auto")

In [ ]:
# inference
def do_inference(model, processor, prompts, max_new_token=50):
  tokenizer = processor.tokenizer
  bad_words = ["<image>","<fake_token_around_image>"]
  if len(bad_words) > 0:
    bad_word_ids = tokenizer(bad_words, add_special_tokens=False).input_ids
  eos_token = "</s>"
  eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)
  inputs = processor(prompts, return_tensors="pt").to(device)

  generated_ids = model.generate(
      **inputs,
      max_new_tokens=max_new_token,
      bad_words_ids=bad_word_ids,
      eos_token_id=[eos_token_id],
      early_stopping=True,
  )
  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
  print(  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0])
  return generated_text

In [ ]:
#preprocessing
def conver_to_rgb(image):